In [1]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from keras.utils import to_categorical
from keras.datasets import fashion_mnist

# Cargar los datos de Fashion MNIST
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Preprocesamiento de los datos
x_train = x_train.reshape(-1, 28, 28, 1) / 255.0
x_test = x_test.reshape(-1, 28, 28, 1) / 255.0
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Construir el modelo de Red Neuronal Profunda
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

# Compilar el modelo
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

with tf.device('/gpu:0'):
	# Entrenar el modelo
	model.fit(x_train, y_train, batch_size=128, epochs=10, validation_data=(x_test, y_test))

	# Evaluar el modelo en el conjunto de prueba
	score = model.evaluate(x_test, y_test, verbose=0)

print('Accuracy en el conjunto de prueba:', score[1])


Epoch 1/10
469/469 [==============================] - 6s 4ms/step - loss: 0.4516 - accuracy: 0.8417 - val_loss: 0.3341 - val_accuracy: 0.8813
Epoch 2/10
469/469 [==============================] - 2s 4ms/step - loss: 0.3072 - accuracy: 0.8900 - val_loss: 0.3126 - val_accuracy: 0.8851
Epoch 3/10
469/469 [==============================] - 2s 4ms/step - loss: 0.2690 - accuracy: 0.9025 - val_loss: 0.2762 - val_accuracy: 0.8986
Epoch 4/10
469/469 [==============================] - 2s 4ms/step - loss: 0.2407 - accuracy: 0.9121 - val_loss: 0.2744 - val_accuracy: 0.8984
Epoch 5/10
469/469 [==============================] - 2s 4ms/step - loss: 0.2228 - accuracy: 0.9175 - val_loss: 0.2747 - val_accuracy: 0.9007
Epoch 6/10
469/469 [==============================] - 2s 4ms/step - loss: 0.2047 - accuracy: 0.9238 - val_loss: 0.2558 - val_accuracy: 0.9083
Epoch 7/10
469/469 [==============================] - 2s 4ms/step - loss: 0.1871 - accuracy: 0.9312 - val_loss: 0.2445 - val_accuracy: 0.9126
Epoch 

In [22]:
import tensorflow as tf
from keras.applications import VGG16
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.datasets import fashion_mnist
from keras.utils import to_categorical
from keras.optimizers import Adam
import numpy as np

# Cargar los datos de Fashion MNIST
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Preprocesamiento de los datos
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

# Reformar los datos de entrada
x_train = np.repeat(x_train[..., np.newaxis], 3, -1)
x_test = np.repeat(x_test[..., np.newaxis], 3, -1)
x_train = np.pad(x_train, ((0, 0), (2, 2), (2, 2), (0, 0)), mode='constant')
x_test = np.pad(x_test, ((0, 0), (2, 2), (2, 2), (0, 0)), mode='constant')

# Cargar el modelo VGG16 preentrenado sin las capas superiores (totalmente conectadas)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Crear un nuevo modelo y agregar el modelo base VGG16 preentrenado
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(10, activation='softmax'))

# 'Congelar' las capas preentrenadas
for layer in base_model.layers:
	layer.trainable = False

# Compilar el modelo
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

with tf.device('/gpu:0'):
	# Entrenar el modelo
	model.fit(x_train, y_train, batch_size=128, epochs=10, validation_data=(x_test, y_test))

	# Evaluar el modelo en el conjunto de prueba
	score = model.evaluate(x_test, y_test, verbose=0)

print('Accuracy en el conjunto de prueba:', score[1])


Epoch 1/10
469/469 [==============================] - 7s 14ms/step - loss: 0.6065 - accuracy: 0.7893 - val_loss: 0.4738 - val_accuracy: 0.8270
Epoch 2/10
469/469 [==============================] - 6s 14ms/step - loss: 0.4272 - accuracy: 0.8432 - val_loss: 0.4256 - val_accuracy: 0.8449
Epoch 3/10
469/469 [==============================] - 6s 14ms/step - loss: 0.3904 - accuracy: 0.8568 - val_loss: 0.4033 - val_accuracy: 0.8530
Epoch 4/10
469/469 [==============================] - 6s 14ms/step - loss: 0.3683 - accuracy: 0.8648 - val_loss: 0.3947 - val_accuracy: 0.8594
Epoch 5/10
469/469 [==============================] - 6s 14ms/step - loss: 0.3511 - accuracy: 0.8714 - val_loss: 0.3865 - val_accuracy: 0.8576
Epoch 6/10
469/469 [==============================] - 6s 13ms/step - loss: 0.3368 - accuracy: 0.8758 - val_loss: 0.3770 - val_accuracy: 0.8642
Epoch 7/10
469/469 [==============================] - 6s 13ms/step - loss: 0.3257 - accuracy: 0.8798 - val_loss: 0.3759 - val_accuracy: 0.8628

In [23]:
from keras.applications import VGG16
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.datasets import fashion_mnist
from keras.utils import to_categorical
from keras.optimizers import Adam
import numpy as np

# Cargar los datos de Fashion MNIST
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Preprocesamiento de los datos
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

# Reformar los datos de entrada
x_train = np.repeat(x_train[..., np.newaxis], 3, -1)
x_test = np.repeat(x_test[..., np.newaxis], 3, -1)
x_train = np.pad(x_train, ((0, 0), (2, 2), (2, 2), (0, 0)), mode='constant')
x_test = np.pad(x_test, ((0, 0), (2, 2), (2, 2), (0, 0)), mode='constant')

# Cargar el modelo VGG16 preentrenado sin las capas superiores (totalmente conectadas)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Crear un nuevo modelo y agregar el modelo base VGG16 preentrenado
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(10, activation='softmax'))

# 'Congelar' las capas preentrenadas
for layer in base_model.layers:
	layer.trainable = False

# Compilar el modelo
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

with tf.device('/gpu:0'):
	# Entrenar el modelo
	model.fit(x_train, y_train, batch_size=128, epochs=10, validation_data=(x_test, y_test))

# Descongelar el último bloque convolucional del modelo VGG16
for layer in model.layers[0].layers[:-4]:
	layer.trainable = True

# Recompilar el modelo después de descongelar algunas capas.
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001), metrics=['accuracy'])

with tf.device('/gpu:0'):
	# Afinar el modelo
	model.fit(x_train, y_train, batch_size=128, epochs=10, validation_data=(x_test, y_test))
	score = model.evaluate(x_test, y_test, verbose=0)

print('Accuracy en el conjunto de prueba:', score[1])


c:\Users\Aphrodite\anaconda3\envs\tf\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/10
469/469 [==============================] - 7s 14ms/step - loss: 0.6073 - accuracy: 0.7909 - val_loss: 0.4765 - val_accuracy: 0.8267
Epoch 2/10
469/469 [==============================] - 6s 13ms/step - loss: 0.4317 - accuracy: 0.8425 - val_loss: 0.4324 - val_accuracy: 0.8432
Epoch 3/10
469/469 [==============================] - 7s 14ms/step - loss: 0.3919 - accuracy: 0.8563 - val_loss: 0.4072 - val_accuracy: 0.8528
Epoch 4/10
469/469 [==============================] - 7s 14ms/step - loss: 0.3697 - accuracy: 0.8632 - val_loss: 0.3927 - val_accuracy: 0.8570
Epoch 5/10
469/469 [==============================] - 6s 14ms/step - loss: 0.3496 - accuracy: 0.8720 - val_loss: 0.3823 - val_accuracy: 0.8621
Epoch 6/10
469/469 [==============================] - 6s 12ms/step - loss: 0.3388 - accuracy: 0.8759 - val_loss: 0.3707 - val_accuracy: 0.8665
Epoch 7/10
469/469 [==============================] - 6s 12ms/step - loss: 0.3267 - accuracy: 0.8791 - val_loss: 0.3732 - val_accuracy: 0.8640